Careless responder feature viz and exploration

In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
from itertools import groupby
import datetime as dt
import scipy as sp
import math

#visualizing results
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import silhouette_score

In [ ]:
data_path = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/TILES/final_data/surveys_CR.pkl'

In [ ]:
#read in csv containing data from all surveys
full_data = pd.read_pickle(data_path)
full_data = pd.DataFrame(data = full_data)
full_data.reset_index(inplace=True, drop=True)

print('Original data shape:\n', full_data.shape, '\n')
#ensure no replicate ID (212 participants in study)
print('Original data unique IDs:\n', full_data['ParticipantID'].unique().shape, '\n')
#ensure no replicate ID (212 participants in study)
print('Original data unique IDs:\n', full_data['MitreID'].unique().shape, '\n')
#how much missing data is there?
print('Original data missing value counts:\n', full_data.isnull().sum(), '\n')
#what is the data type of each column?
print('Original data data types:\n', full_data.info(), '\n')

In [ ]:
full_data['survey_type'].value_counts()

In [ ]:
#should be 71
len(full_data['wave_study_day'].unique())

In [ ]:
full_data.head(1)

### explore CR across survey types

In [ ]:
full_data['survey_type'].value_counts()

In [ ]:
CR_features = ['context_homevsworking',
       'context_workvsactivities', 'context_workvswork',
       'context_drivevsdrive', 'mean_diff_hinderance_vs_support',
       'longstring_count_engage', 'longstring_answer_engage',
       'longstring_count_norm_engage', 'longstring_mult_engage',
       'ls_auc_engage', 'ls_std_engage', 'ls_skew_engage',
       'ls_kurt_engage', 'ls_auc_seeded_engage', 'ls_std_seeded_engage',
       'ls_skew_seeded_engage', 'ls_kurt_seeded_engage', 'mean_diff_ave',
       'longstring_count_norm_ave', 'longstring_count_pf',
       'longstring_answer_pf', 'longstring_count_norm_pf',
       'longstring_mult_pf', 'ls_auc_pf', 'ls_std_pf', 'ls_skew_pf',
       'ls_kurt_pf', 'ls_auc_seeded_pf', 'ls_std_seeded_pf',
       'ls_skew_seeded_pf', 'ls_kurt_seeded_pf',
       'mean_diff_affect_posneg', 'mean_diff_nervous_anxiety',
       'mean_diff_irb_irb', 'mean_diff_dalal_posneg',
       'longstring_count_affect', 'longstring_answer_affect',
       'longstring_count_norm_affect', 'longstring_mult_affect',
       'ls_auc_affect', 'ls_std_affect', 'ls_skew_affect',
       'ls_kurt_affect', 'ls_auc_seeded_affect', 'ls_std_seeded_affect',
       'ls_skew_seeded_affect', 'ls_kurt_seeded_affect',
       'longstring_count_irbd', 'longstring_answer_irbd',
       'longstring_count_norm_irbd', 'longstring_mult_irbd',
       'ls_auc_irbd', 'ls_std_irbd', 'ls_skew_irbd', 'ls_kurt_irbd',
       'ls_auc_seeded_irbd', 'ls_std_seeded_irbd', 'ls_skew_seeded_irbd',
       'ls_kurt_seeded_irbd', 'longstring_count_dalal',
       'longstring_answer_dalal', 'longstring_count_norm_dalal',
       'longstring_mult_dalal', 'ls_auc_dalal', 'ls_std_dalal',
       'ls_skew_dalal', 'ls_kurt_dalal', 'ls_auc_seeded_dalal',
       'ls_std_seeded_dalal', 'ls_skew_seeded_dalal',
       'ls_kurt_seeded_dalal', 'intake_check_alc',
       'intake_check_nic', 'mean_diff_extraversion',
       'mean_diff_agreeableness', 'mean_diff_conscientiousness',
       'mean_diff_neuroticism', 'mean_diff_openness',
       'longstring_count_personality', 'longstring_answer_personality',
       'longstring_count_norm_personality', 'longstring_mult_personality',
       'ls_auc_personality', 'ls_std_personality', 'ls_skew_personality',
       'ls_kurt_personality', 'ls_auc_seeded_personality',
       'ls_std_seeded_personality', 'ls_skew_seeded_personality',
       'ls_kurt_seeded_personality', 'time_to_complete']

In [ ]:
diff_cols = ['mean_diff_hinderance_vs_support', 'mean_diff_affect_posneg',
       'mean_diff_nervous_anxiety', 'mean_diff_irb_irb',
       'mean_diff_dalal_posneg', 'mean_diff_extraversion', 'mean_diff_agreeableness', 
        'mean_diff_conscientiousness', 'mean_diff_neuroticism', 'mean_diff_openness']

longstring_norm_cols = ['longstring_count_norm_engage', 
                        'longstring_count_norm_ave', 
                        'longstring_count_norm_pf',
                        'longstring_count_norm_affect', 
                        'longstring_count_norm_irbd',
                        'longstring_count_norm_dalal',
                        'longstring_count_norm_personality']


In [ ]:
shared_cols = ['context_homevsworking', 'context_workvsactivities', 'context_workvswork', 'context_drivevsdrive', 
               'mean_diff_ave', 'mean_diff_affect_posneg', 'mean_diff_nervous_anxiety',
               'longstring_count_norm_ave', 'longstring_count_norm_affect', 'longstring_count_norm_ave',
               'ls_auc_affect', 'ls_std_affect', 'ls_skew_affect', 'ls_kurt_affect', 
               'ls_auc_seeded_affect', 'ls_std_seeded_affect', 'ls_skew_seeded_affect', 'ls_kurt_seeded_affect']

i=1
plt.figure(figsize=(50,30))
grid_size = math.ceil(np.sqrt(len(shared_cols)))
for feature in shared_cols:
    plt.subplot(grid_size, grid_size, i)
    sns.barplot(x='survey_type', y=feature, data=full_data)
    i = i + 1

In [ ]:
#create new df with normalized mean_diff values
survey_CR_normdiff = pd.melt(full_data, id_vars=['survey_type', 'wave_study_date_bin'],  value_vars=['mean_diff_hinderance_vs_support', 'mean_diff_affect_posneg',
       'mean_diff_nervous_anxiety', 'mean_diff_irb_irb',
       'mean_diff_dalal_posneg', 'mean_diff_extraversion', 'mean_diff_agreeableness', 
        'mean_diff_conscientiousness', 'mean_diff_neuroticism', 'mean_diff_openness'])
survey_CR_normdiff.head()

In [ ]:
sns.catplot(x="wave_study_date_bin", y="value", hue='variable', data=survey_CR_normdiff, kind='bar', height=10, col='survey_type')

In [ ]:
#create new df with normalized longstring values
survey_CR_normls = pd.melt(full_data, id_vars=['survey_type', 'wave_study_date_bin'],  value_vars=['longstring_count_norm_engage', 
                        'longstring_count_norm_ave', 
                        'longstring_count_norm_pf',
                        'longstring_count_norm_affect', 
                        'longstring_count_norm_irbd',
                        'longstring_count_norm_dalal',
                        'longstring_count_norm_personality'])
survey_CR_normls.head()


In [ ]:
plt.figure(figsize=(30,30))
sns.catplot(x="wave_study_date_bin", y="value", hue='variable', data=survey_CR_normls, kind='bar', height=7, col='survey_type')

In [ ]:
participants = full_data['MitreID'].unique()

for part in participants:
    data_part = full_data[full_data['MitreID'] == part]
    sns.catplot(x='wave_study_date_bin', y='longstring_count_norm_ave', data=data_part, ci=None, col='survey_type', kind='bar')
    plt.show()

In [ ]:
participants = full_data['MitreID'].unique()

for part in participants:
    data_part = full_data[full_data['MitreID'] == part]
    sns.catplot(x='wave_study_day', y='mean_diff_ave', data=data_part, ci=None, col='survey_type', kind='bar')
    plt.show()

In [ ]:
sns.catplot(x='MitreID', y='mean_diff_ave', data=full_data, col='wave_study_date_bin', kind='bar', col_wrap=2, height=10, ci=None)

In [ ]:
sns.catplot(x='MitreID', y='longstring_count_norm_ave', data=full_data, col='wave_study_date_bin', kind='bar', col_wrap=2, height=10, ci=None)

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(x='wave_study_day', y='mean_diff_affect_posneg', data=full_data, hue='survey_type', ci=None)

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(x='wave_study_day', y='mean_diff_nervous_anxiety', data=full_data, hue='survey_type', ci=None)

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(x='wave_study_day', y='longstring_count_norm_affect', data=full_data, hue='survey_type', ci=None)

In [ ]:
full_data.head()

In [ ]:
engage_CRs = ['mean_diff_hinderance_vs_support', 'time_to_complete',
       'longstring_count_engage', 'longstring_answer_engage',
       'longstring_count_norm_engage', 'longstring_mult_engage',
       'ls_auc_engage', 'ls_std_engage', 'ls_skew_engage',
       'ls_kurt_engage', 'ls_auc_seeded_engage', 'ls_std_seeded_engage',
       'ls_skew_seeded_engage', 'ls_kurt_seeded_engage', 'context_homevsworking',
       'context_workvsactivities', 'context_workvswork',
       'context_drivevsdrive']

psych_flex_CRs = ['longstring_count_pf',
       'longstring_answer_pf', 'longstring_count_norm_pf',
       'longstring_mult_pf', 'ls_auc_pf', 'ls_std_pf', 'ls_skew_pf',
       'ls_kurt_pf', 'ls_auc_seeded_pf', 'ls_std_seeded_pf',
       'ls_skew_seeded_pf', 'ls_kurt_seeded_pf', 'time_to_complete']

jobsatwork_CRs = ['mean_diff_ave',
       'mean_diff_affect_posneg', 'mean_diff_nervous_anxiety',
       'mean_diff_irb_irb', 'mean_diff_dalal_posneg', 
                  'longstring_count_norm_ave', 'longstring_count_norm_affect', 'longstring_count_norm_irbd', 'longstring_count_norm_dalal',
                  'time_to_complete',
       'longstring_count_affect', 'longstring_answer_affect',
        'longstring_mult_affect',
       'ls_auc_affect', 'ls_std_affect', 'ls_skew_affect',
       'ls_kurt_affect', 'ls_auc_seeded_affect', 'ls_std_seeded_affect',
       'ls_skew_seeded_affect', 'ls_kurt_seeded_affect',
       'longstring_count_irbd', 'longstring_answer_irbd',
       'longstring_mult_irbd',
       'ls_auc_irbd', 'ls_std_irbd', 'ls_skew_irbd', 'ls_kurt_irbd',
       'ls_auc_seeded_irbd', 'ls_std_seeded_irbd', 'ls_skew_seeded_irbd',
       'ls_kurt_seeded_irbd', 'longstring_count_dalal',
       'longstring_answer_dalal', 
       'longstring_mult_dalal', 'ls_auc_dalal', 'ls_std_dalal',
       'ls_skew_dalal', 'ls_kurt_dalal', 'ls_auc_seeded_dalal',
       'ls_std_seeded_dalal', 'ls_skew_seeded_dalal',
       'ls_kurt_seeded_dalal', 'context_homevsworking',
       'context_workvsactivities', 'context_workvswork',
       'context_drivevsdrive']

jobsnotatwork_CRs = ['mean_diff_ave',
       'mean_diff_affect_posneg', 'mean_diff_nervous_anxiety',
       'longstring_count_affect', 'longstring_answer_affect',
       'longstring_count_norm_affect', 'longstring_mult_affect',
       'ls_auc_affect', 'ls_std_affect', 'ls_skew_affect',
       'ls_kurt_affect', 'ls_auc_seeded_affect', 'ls_std_seeded_affect',
       'ls_skew_seeded_affect', 'ls_kurt_seeded_affect', 'time_to_complete', 'context_homevsworking',
       'context_workvsactivities', 'context_workvswork',
       'context_drivevsdrive']

health_CRs = ['mean_diff_ave',
       'mean_diff_affect_posneg', 'mean_diff_nervous_anxiety', 'intake_check_alc',
       'intake_check_nic', 'time_to_complete',
       'longstring_count_affect', 'longstring_answer_affect',
       'longstring_count_norm_affect', 'longstring_mult_affect',
       'ls_auc_affect', 'ls_std_affect', 'ls_skew_affect',
       'ls_kurt_affect', 'ls_auc_seeded_affect', 'ls_std_seeded_affect',
       'ls_skew_seeded_affect', 'ls_kurt_seeded_affect', 'context_homevsworking',
       'context_workvsactivities', 'context_workvswork',
       'context_drivevsdrive']

personality_CRs = ['mean_diff_ave',
       'mean_diff_affect_posneg', 'mean_diff_nervous_anxiety', 'mean_diff_extraversion',
       'mean_diff_agreeableness', 'mean_diff_conscientiousness',
       'mean_diff_neuroticism', 'mean_diff_openness', 
                   'longstring_count_norm_ave', 'longstring_count_norm_affect', 'longstring_count_norm_personality',
                   'time_to_complete', 
       'longstring_count_affect', 'longstring_answer_affect',
       'longstring_mult_affect',
       'ls_auc_affect', 'ls_std_affect', 'ls_skew_affect',
       'ls_kurt_affect', 'ls_auc_seeded_affect', 'ls_std_seeded_affect',
       'ls_skew_seeded_affect', 'ls_kurt_seeded_affect', 
       'longstring_count_personality', 'longstring_answer_personality',
        'longstring_mult_personality',
       'ls_auc_personality', 'ls_std_personality', 'ls_skew_personality',
       'ls_kurt_personality', 'ls_auc_seeded_personality',
       'ls_std_seeded_personality', 'ls_skew_seeded_personality',
       'ls_kurt_seeded_personality', 'context_homevsworking',
       'context_workvsactivities', 'context_workvswork',
       'context_drivevsdrive']

In [ ]:
engage = full_data[full_data['survey_type'] == 'engage_psycap'][engage_CRs]

print(engage.isna().sum())

engage_CRs_corr = engage.corr()
engage_CRs_corr.to_csv('engage_corr.csv')
engage_CRs_corr
#sns.pairplot(engage, kind='reg', diag_kind="kde")

In [ ]:
psych_flex = full_data[full_data['survey_type'] == 'psych_flex'][psych_flex_CRs]

print(psych_flex.isna().sum())

#sns.pairplot(psych_flex, kind='reg', diag_kind="kde")

pf_CRs_corr = psych_flex.corr()
pf_CRs_corr.to_csv('pf_CRs_corr.csv')
pf_CRs_corr

In [ ]:
jobs_atwork = full_data[full_data['survey_type'] == 'jobs_atwork'][jobsatwork_CRs]

print(jobs_atwork.isna().sum())

#sns.pairplot(jobs_atwork, kind='reg', diag_kind="kde")

jobs_atwork_corr = jobs_atwork.corr()
jobs_atwork_corr.to_csv('jobs_atwork_corr.csv')
jobs_atwork_corr

In [ ]:
jobs_notatwork = full_data[full_data['survey_type'] == 'jobs_notatwork'][jobsnotatwork_CRs]

print(jobs_notatwork.isna().sum())

#sns.pairplot(jobs_atwork, kind='reg', diag_kind="kde")

jobs_notatwork_corr = jobs_notatwork.corr()
jobs_notatwork_corr.to_csv('jobs_notatwork_corr.csv')
jobs_notatwork_corr

In [ ]:
health = full_data[full_data['survey_type'] == 'health'][health_CRs]

print(health.isna().sum())

#sns.pairplot(jobs_atwork, kind='reg', diag_kind="kde")

health_corr = health.corr()
health_corr.to_csv('health_corr.csv')
health_corr

In [ ]:
personality = full_data[full_data['survey_type'] == 'personality'][personality_CRs]

print(personality.isna().sum())

#sns.pairplot(jobs_atwork, kind='reg', diag_kind="kde")

personality_corr = personality.corr()
personality_corr.to_csv('personality_corr.csv')
personality_corr

In [ ]:
i=1
plt.figure(figsize=(30,20))
grid_size = math.ceil(np.sqrt(len(engage_CRs)))
for feature in engage_CRs:
    plt.subplot(grid_size, grid_size, i)
    sns.distplot(engage[feature].dropna(axis=0))
    i = i + 1


In [ ]:
i=1
plt.figure(figsize=(30,20))
grid_size = math.ceil(np.sqrt(len(psych_flex_CRs)))
for feature in psych_CRs[1:]:
    plt.subplot(grid_size, grid_size, i)
    sns.distplot(psych_flex[feature].dropna())
    i = i + 1


In [ ]:
i=1
plt.figure(figsize=(30,30))
grid_size = math.ceil(np.sqrt(len(jobsatwork_CRs)))
for feature in jobsatwork_CRs[1:]:
    plt.subplot(grid_size, grid_size, i)
    sns.distplot(jobs_atwork[feature].dropna())
    i = i + 1


In [ ]:
i=1
plt.figure(figsize=(30,30))
grid_size = math.ceil(np.sqrt(len(health_CRs)))
for feature in health_CRs[1:]:
    plt.subplot(grid_size, grid_size, i)
    sns.distplot(health[feature].dropna())
    i = i + 1


In [ ]:
i=1
plt.figure(figsize=(30,30))
grid_size = math.ceil(np.sqrt(len(personality_CRs)))
for feature in personality_CRs[1:]:
    plt.subplot(grid_size, grid_size, i)
    sns.distplot(personality[feature].dropna())
    i = i + 1
